In [1]:
import requests

from sitemap import parse_sitemap
from api import MediaWikiSession, MediaWikiAPI
from cachecontrol import CacheControl

In [2]:
req_session = CacheControl(requests.session())

wb = MediaWikiAPI(MediaWikiSession("de.wikibooks.org", req_session))
wp = MediaWikiAPI(MediaWikiSession("de.wikipedia.org", req_session))

In [14]:
sitemap_code = wb.get_content("Mathe für Nicht-Freaks: Sitemap")
sitemap = parse_sitemap(sitemap_code)
sitemap["name"] = "Mathe für Nicht-Freaks"

In [37]:
def fmap(func):
    def fmap_func(node):
        result = {"name": node["name"]}
        
        if "link" in node and node["link"]:
            result["result"] = func(node["link"])
        else:
            result["result"] = None
        
        result["children"] = [fmap_func(child) for child in node["children"]]
        
        return result
    return fmap_func

def report(func, aggr = lambda node: node["result"]):
    def showreport(node, indent = 0):
        line = " " * (indent*4)
        line += "* "
        line += node["name"]

        result = aggr(node)

        if result != None:
            line += " – "
            line += str(result)

        print(line)
            
        for child in node["children"]:
            showreport(child, indent+1)
        
    
    return (lambda node: showreport(fmap(func)(node)))

def sum_aggr(node):
    views = sum((views2016aggr(x) for x in node["children"]))
    
    if node["result"]:
        views += node["result"]
    
    return views

# Seitenaufrufe 2016

In [33]:
def views2016(x):
    return wb.pageviews(x, "20160101", "20161231")

report(views2016, sum_aggr)(sitemap)

* Mathe für Nicht-Freaks – 1576191
    * Grundlagen der Mathematik – 805988
        * Was ist Mathematik? – 5232
            * Was ist Mathematik? – 5225
            * Teilgebiete der Mathematik – 7
        * Einführung in die Logik – 127592
            * Aussagenlogik – 13090
            * Junktoren – 21300
            * Notwendige und hinreichende Bedingungen – 5002
            * Wahrheitstabelle – 12370
            * Quantoren – 23060
            * Aussageform und Substitution – 3120
            * Tautologien – 10559
            * Aussagen formalisieren – 4340
            * Aussagen negieren – 28804
            * Gesetze der Logik – 3697
            * Aufgaben – 2250
        * Beweise und Beweismethoden – 43512
            * Was sind Beweise? – 10394
            * Direkter und indirekter Beweis – 23268
            * Fallunterscheidung und Kontraposition – 9850
        * Vollständige Induktion – 109156
            * Definition und Erklärung – 76878
            * Beispielaufgaben – 32

# Zuwachs Seitenaufrufe zwischen WiSe2016/17 zu WiSe2015/16

In [36]:
def view_diff(x):
    return wb.pageviews(x, "20161001", "20170331")-wb.pageviews(x, "20151001", "20160331")

report(view_diff, sum_aggr)(sitemap)

* Mathe für Nicht-Freaks – 355071
    * Grundlagen der Mathematik – 38056
        * Was ist Mathematik? – 318
            * Was ist Mathematik? – -179
            * Teilgebiete der Mathematik – 497
        * Einführung in die Logik – 11760
            * Aussagenlogik – -1377
            * Junktoren – -2841
            * Notwendige und hinreichende Bedingungen – -123
            * Wahrheitstabelle – 1479
            * Quantoren – 1523
            * Aussageform und Substitution – -342
            * Tautologien – 960
            * Aussagen formalisieren – 558
            * Aussagen negieren – 10609
            * Gesetze der Logik – 1560
            * Aufgaben – -246
        * Beweise und Beweismethoden – 3903
            * Was sind Beweise? – 15
            * Direkter und indirekter Beweis – 3676
            * Fallunterscheidung und Kontraposition – 212
        * Vollständige Induktion – -7784
            * Definition und Erklärung – 1218
            * Beispielaufgaben – -9002
        * M

In [60]:
def wikipedia_title(x):
    params = {
        "action":"wbgetentities",
        "sites":"dewikibooks",
        "titles":x,
        "props":"sitelinks",
        "languages":"nn",
        "format":"json"
    }
    
    result = req_session.get("http://wikidata.org/w/api.php", params=params).json()["entities"]
    key, value = result.popitem()
    
    if key == "-1":
        return None
    else:
        value = value["sitelinks"]
        
        if "dewiki" in value:
            return value["dewiki"]["title"]
        else:
            return None

def wikipedia_share(article):
    wiki_title = wikipedia_title(article)
    
    if wiki_title:
        return wb.pageviews(article, "20161001", "20170331") / wp.pageviews(wiki_title, "20161001", "20170331")
    else:
        return None

report(wikipedia_share)(sitemap)

* Mathe für Nicht-Freaks
    * Grundlagen der Mathematik – 6.544865642994242
        * Was ist Mathematik?
            * Was ist Mathematik? – 0.02206980293060729
            * Teilgebiete der Mathematik
        * Einführung in die Logik
            * Aussagenlogik
            * Junktoren
            * Notwendige und hinreichende Bedingungen
            * Wahrheitstabelle
            * Quantoren
            * Aussageform und Substitution
            * Tautologien
            * Aussagen formalisieren
            * Aussagen negieren
            * Gesetze der Logik
            * Aufgaben
        * Beweise und Beweismethoden
            * Was sind Beweise?
            * Direkter und indirekter Beweis
            * Fallunterscheidung und Kontraposition
        * Vollständige Induktion
            * Definition und Erklärung – 0.7716754830132104
            * Beispielaufgaben
        * Mengenlehre
            * Definition einer Menge
            * Mengenschreibweisen
            * Russells An